In [43]:
the_sum = 0
with open("input12.dat", 'r') as f:
    #reading file line
    content = f.readline().removesuffix('\n').split()
    while content != []:
        string, nums = content[0], [int(_) for _ in content[1].split(sep=',')]
        #creating list of strings with all possible subs
        str_list = [string]
        while '?' in str_list[-1]:
            for i in range(len(str_list)):
                str_list.append(str_list[i].replace("?", "#", 1))
                str_list[i] = str_list[i].replace("?", ".", 1)
        #testing how many possibilities match the numbers
        the_sum = the_sum + [[len(_) for _ in string.split(sep=".") if _ != ""] for string in str_list].count(nums)
        #reading file line in loop    
        content = f.readline().removesuffix('\n').split()
the_sum

7792

In [301]:
#very recursive function

cahce = {} #to memorize and avoid recalculating stuff; absurdly faster

def count(string, nums):
    if string == "":
        return 1 if nums == () else 0
    if nums == ():
        return 0 if "#" in string else 1
    
    key = (string, nums)
    if key in cahce:
        return cahce[key]
    
    result = 0
    if string[0] in ".?":
        result += count(string[1:], nums)
    if string[0] in "#?":
        #if block is valid
        if nums[0] <= len(string) and "." not in string[:nums[0]] and (nums[0]==len(string) or string[nums[0]]!="#"):
            result += count(string[nums[0]+1:], nums[1:])
    
    cahce[key] = result
    return result

the_sum = 0
with open("input12.dat", 'r') as f:
    lines = f.readlines()
for line in lines:
    content = line.removesuffix('\n').split()
    string, nums = content[0], tuple(map(int, content[1].split(sep=',')))
    the_sum = the_sum + count(string, nums)
the_sum

7792

In [366]:
the_sum = 0
with open("input12.dat", 'r') as f:
    lines = f.readlines()
results = []
for line in lines:
    # print("\r"+line.removesuffix('\n'), end="")
    content = line.removesuffix('\n').split()
    string, nums = "?".join([content[0]]*5), tuple(map(int,content[1].split(sep=',')))*5
    results.append(count(string, nums))
    the_sum = the_sum + count(string, nums)
the_sum

13012052341533

In [346]:
#with functions
def processround(string, nums, absent=None):
    if absent is None:
        absent = [n for n in range(1, max(nums)) if n not in nums]
    string = string.replace("?"+"#"*max(nums),"."+"#"*max(nums))
    string = string.replace("#"*max(nums)+"?","#"*max(nums)+".")
    for n in absent:
        string = string.replace("?"+"#"*n+".","#"*(n+1)+".")
        string = string.replace("."+"#"*n+"?","."+"#"*(n+1))
    for n in range(1,min(nums)):
        string = string.replace("."+"?"*n+".", "."*(n+2))
    return string

def midcount_bydot(mid_str, nums_str, nums=None, midprocessing=False, killtest=False):
    mid_possibs = []
    str_list = [mid_str]
    if nums is None:
        nums = tuple(map(int,nums_str.split(",")))
    while str_list != []: #list emptied when items have no "?" anymore
        string = str_list[-1]
        str_list.remove(string) 

        if midprocessing:
            string = processround(string, nums)
        if killtest:#kill a string with an impossible number of # and ?
            if string.count("#") > the_sum(nums) or string.count("#")+string.count("?") < the_sum(nums):
                continue 

        test_set = {string.replace("?", ".", 1), string.replace("?", "#", 1)}
        for test_string in test_set:
            if "?" not in test_string:
                len_tuple = tuple(map(len,test_string.replace("."," ").split()))
                if str(len_tuple).strip("(),").replace(" ","") in nums_str:
                    mid_possibs.append(len_tuple)
                continue
            else:
                len_tuple = tuple(map(len,test_string[:test_string.find("?")].replace("."," ").split()))
                len_str = str(len_tuple).strip("(),").replace(" ","")
                if len_str in nums_str or (test_string[:test_string.find("?")][-1]=="#" and len_str[:-1] in nums_str):
                        str_list.append(test_string)

    return {sequence:mid_possibs.count(sequence) for sequence in mid_possibs}

def startcount_bydot(start_str, nums, midprocessing=False, killtest=False):
    start_possibs = []
    str_list = [start_str]
    while str_list != []: #list emptied when items have no "?" anymore
        string = str_list[-1]
        #remove old string
        str_list.remove(string)       
        if midprocessing:
            string = processround(string, list[nums])
        if killtest:#kill a string with an impossible number of # and ?
            if string.count("#") > sum(nums) or string.count("#")+string.count("?") < sum(nums):
                continue 

        test_set = {string.replace("?", ".", 1), string.replace("?", "#", 1)}
        for test_string in test_set:
            if "?" not in test_string:
                len_tuple = tuple(map(len,test_string.replace("."," ").split()))
                if len_tuple == nums[:len(len_tuple)]:
                    start_possibs.append(len_tuple)
                continue
            else:
                len_tuple = tuple(map(len,test_string[:test_string.find("?")].replace("."," ").split()))
                if len_tuple == nums[:len(len_tuple)] or (test_string[:test_string.find("?")][-1]=="#" and len_tuple[:-1] == nums[:len(len_tuple)][:-1] and len_tuple[-1] <= nums[len(len_tuple)-1]):
                    str_list.append(test_string)

    return {sequence:start_possibs.count(sequence) for sequence in start_possibs} 

def count_broken_by_dots(string, nums_str):#nums_str
    nums = tuple(map(int,nums_str.split(",")))
    #preprocessing on string
    string = processround(string, nums)
    start_str, end_str = string[:string.find(".")], string[string.find(".")+1:]
    mid_str = end_str + "?" + start_str #this one happens 4 times

    mid_counts_of   = midcount_bydot(mid_str, nums_str, nums)
    start_counts_of = startcount_bydot(start_str, nums)
    end_counts_of   = startcount_bydot(end_str[::-1], nums[::-1])
    end_counts_of   = {key[::-1]:end_counts_of[key] for key in end_counts_of.keys()}

    #there should be a cleaner way
    the_sum = 0
    for s_seq in start_counts_of:
        for m_seq1 in mid_counts_of:
            if s_seq+m_seq1 == nums[:len(s_seq+m_seq1)]:
                for m_seq2 in mid_counts_of:
                    if s_seq+m_seq1+m_seq2 == nums[:len(s_seq+m_seq1+m_seq2)]:
                        for m_seq3 in mid_counts_of:
                            if s_seq+m_seq1+m_seq2+m_seq3 == nums[:len(s_seq+m_seq1+m_seq2+m_seq3)]:
                                for m_seq4 in mid_counts_of:
                                    if s_seq+m_seq1+m_seq2+m_seq3+m_seq4 == nums[:len(s_seq+m_seq1+m_seq2+m_seq3+m_seq4)]:
                                        for e_seq in end_counts_of:
                                                if s_seq+m_seq1+m_seq2+m_seq3+m_seq4+e_seq == nums:
                                                    the_sum = the_sum + start_counts_of[s_seq]*mid_counts_of[m_seq1]*mid_counts_of[m_seq2]*mid_counts_of[m_seq3]*mid_counts_of[m_seq4]*end_counts_of[e_seq]
    return the_sum

def midcount_byttt(mid_str, nums_str, nums=None, midprocessing=False, killtest=False):#assumes it begins on "#"
    mid_possibs = []
    str_list = [mid_str]
    if nums is None:
        nums = tuple(map(int,nums_str.split(",")))
    nums_str = nums_str.replace("10","A").replace("11","B").replace("12","C").replace("13","D").replace("14","E").replace("15","F").replace("16","G")
    while str_list != []: #list emptied when items have no "?" anymore
        string = str_list[-1]
        str_list.remove(string) 

        if midprocessing:
            string = processround(string, nums)
        if killtest:#kill a string with an impossible number of # and ?
            if string.count("#") > the_sum(nums) or string.count("#")+string.count("?") < the_sum(nums):
                continue 

        test_set = {string.replace("?", ".", 1), string.replace("?", "#", 1)}
        for test_string in test_set:
            if "?" not in test_string:
                len_tuple = tuple(map(len,test_string.replace("."," ").split()))
                if max(len_tuple+(0,)) <= max(nums):
                    len_str = str(len_tuple).strip("()").replace(" ","").replace("10","A").replace("11","B").replace("12","C").replace("13","D").replace("14","E").replace("15","F").replace("16","G")
                    if test_string[-1]!="#" and len_str[1:] in nums_str: #leaving the commas in len_str to compare (good)
                        mid_possibs.append(len_tuple)
                    elif test_string[-1]=="#" and len_str[1:-1] in nums_str:
                        mid_possibs.append(len_tuple+("+",))
            else:
                len_tuple = tuple(map(len,test_string[:test_string.find("?")].replace("."," ").split()))
                if max(len_tuple+(0,)) <= max(nums):
                    len_str = str(len_tuple).strip("()").replace(" ","").replace("10","A").replace("11","B").replace("12","C").replace("13","D").replace("14","E").replace("15","F").replace("16","G")
                    if len_str in nums_str or (test_string[0]=="#" and len_str[1:] in nums_str)  or (test_string[:test_string.find("?")][-1]=="#" and len_str[:-1] in nums_str) or (test_string[0]=="#" and test_string[:test_string.find("?")][-1]=="#" and len_str[1:-1] in nums_str):
                            str_list.append(test_string)

    return {sequence:mid_possibs.count(sequence) for sequence in mid_possibs}

def stcount_byttt(start_str, nums, midprocessing=False, killtest=False):
    start_possibs = []
    str_list = [start_str]
    while str_list != []: #list emptied when items have no "?" anymore
        string = str_list[-1]
        #remove old string
        str_list.remove(string)       
        if midprocessing:
            string = processround(string, list[nums])
        if killtest:#kill a string with an impossible number of # and ?
            if string.count("#") > sum(nums) or string.count("#")+string.count("?") < sum(nums):
                continue 

        test_set = {string.replace("?", ".", 1), string.replace("?", "#", 1)}
        for test_string in test_set:
            if "?" not in test_string:
                len_tuple = tuple(map(len,test_string.replace("."," ").split()))
                if len_tuple == nums[:len(len_tuple)] and test_string[-1:]!="#":
                    start_possibs.append(len_tuple)
                elif (test_string[-1]=="#" and len_tuple[:-1] == nums[:len(len_tuple)][:-1] and len_tuple[-1] <= nums[len(len_tuple)-1]):
                    start_possibs.append(len_tuple+("+",))
            else:
                len_tuple = tuple(map(len,test_string[:test_string.find("?")].replace("."," ").split()))
                if len_tuple == nums[:len(len_tuple)] or (test_string[:test_string.find("?")][-1]=="#" and len_tuple[:-1] == nums[:len(len_tuple)][:-1] and len_tuple[-1] <= nums[len(len_tuple)-1]):
                    str_list.append(test_string)

    return {sequence:start_possibs.count(sequence) for sequence in start_possibs} 

def compare(list_seqs, nums, partial_comp=True):#compare partial and full list with the correct concatenation
    current_seq = [it for l in list_seqs for it in l]
    while "+" in current_seq[:-1]:
        i = current_seq.index("+")
        current_seq[i-1] = current_seq[i-1]+current_seq[i+1]
        current_seq[i] = "_"
        current_seq[i+1] = "_"
    current_seq = [it for it in current_seq if it !="_"]
    # print(current_seq, list(nums))
    if partial_comp:
        if current_seq[-1:] == ["+"]:
            current_seq = current_seq[:-2]
        if current_seq == list(nums[:len(current_seq)]):
            return True
        return False
    if current_seq == list(nums):
        return True
    return False

def count_broken_by_ttt(string, nums_str):#assumes at least one "#" in string
    nums = tuple(map(int,nums_str.split(",")))
    start_str = string[:string.find("#")]
    end_str = string[string.find("#"):] #begins in "#" necessarily
    mid_str = end_str + "?" + start_str #this one happens 4 times

    mid_counts_of   = midcount_byttt(mid_str, nums_str, nums)
    start_counts_of = stcount_byttt(start_str, nums)
    end_counts_of   = stcount_byttt(end_str[::-1], nums[::-1])
    end_counts_of   = {key[::-1][1:]:end_counts_of[key] for key in end_counts_of.keys()}
    # start_str, start_counts_of, mid_str, mid_counts_of, end_str, end_counts_of

    # there should be a cleaner way
    the_sum = 0
    for s_seq in start_counts_of:
        for m_seq1 in mid_counts_of:
            # print(s_seq, m_seq1, nums)
            if compare([s_seq, m_seq1], nums):
                # print(s_seq, m_seq1, nums)
                for m_seq2 in mid_counts_of:
                    if compare([s_seq, m_seq1, m_seq2], nums):
                        for m_seq3 in mid_counts_of:
                            if compare([s_seq, m_seq1, m_seq2, m_seq3], nums):
                                for m_seq4 in mid_counts_of:
                                    if compare([s_seq, m_seq1, m_seq2, m_seq3, m_seq4], nums):
                                        for e_seq in end_counts_of:
                                            if compare([s_seq, m_seq1, m_seq2, m_seq3, m_seq4, e_seq], nums, partial_comp=False):
                                                the_sum = the_sum + start_counts_of[s_seq]*mid_counts_of[m_seq1]*mid_counts_of[m_seq2]*mid_counts_of[m_seq3]*mid_counts_of[m_seq4]*end_counts_of[e_seq]

    return the_sum

def count_question_only(string, nums):
    import math
    #with n_dots, out of n_dots+1 spacings around them, n_blocks of those must be chosen to fit a block of # in. Order of choice irrelevant because actual order defined by nums
    n_dots = len(string)-sum(nums)
    n_blocks = len(nums)
    # n_free_dots = n_dots - n_interstitials + 2 #freely available dots, excludes the min of 1 per interstitials (except the end 2)
    return int(math.factorial(n_dots+1)/math.factorial(n_blocks)/math.factorial(n_dots+1-n_blocks))

In [364]:
with open("input12.dat", 'r') as f:
    lines = f.readlines()

total_sum = 0
wr_results = []
print("Start", end="")
for line in lines:
    print("\r"+line.removesuffix('\n'), end="")
    content = line.removesuffix('\n').split()
    string, short_nums = content[0], tuple(map(int,content[1].split(sep=',')))
    string = processround(string, short_nums)
    if "." in string:
        string, nums_str = content[0], ((content[1]+",")*5)[:-1]
        # print(content, count_broken_by_dots(string, nums_str))
        the_count = count_broken_by_dots(string, nums_str)
    elif "#" in string:
        string, nums_str = content[0], ((content[1]+",")*5)[:-1]
        # print(content, count_broken_by_ttt(string, nums_str))
        the_count = count_broken_by_ttt(string, nums_str)
    else:
        string, nums = "?".join([content[0]]*5), tuple(map(int,content[1].split(sep=',')))*5
        # print(content, count_question_only(string, nums))
        the_count = count_question_only(string, nums)
    wr_results.append(the_count)
    total_sum = total_sum + the_count
print("\rThe calculation is now finished!",end="")
total_sum

The calculation is now finished!

13012052341533

In [365]:
([(r,w, wr_results.index(w)) for r,w in zip(results[800:], wr_results) if r!=w])

[(3888, 7776, 0),
 (25688, 1024, 1),
 (3888, 12005, 2),
 (122, 1, 3),
 (1312, 6480, 4),
 (8187, 11481, 5),
 (32, 512, 6),
 (2546251448, 1, 3),
 (57744972, 32, 8),
 (49296737, 1, 3),
 (32, 13645, 10),
 (32, 1, 3),
 (1, 243, 12),
 (243, 1088, 13),
 (60062, 1, 3),
 (4687500, 1024, 1),
 (212824, 32, 8),
 (60000, 162, 17),
 (32, 1738077621, 18),
 (316458, 243, 12),
 (32, 243, 12),
 (9604, 16, 21),
 (118001081, 22666344, 22),
 (207360, 476, 23),
 (49800946, 512, 6),
 (16720, 32, 8),
 (24185787, 3669975, 26),
 (1, 1250, 27),
 (268859, 32, 8),
 (243, 2121670, 29),
 (873242, 32, 8),
 (16384, 24576, 31),
 (3888, 32022325, 32),
 (1605920, 9450, 33),
 (1875, 21419, 34),
 (11525, 47107630, 35),
 (8131549934354, 512, 6),
 (155117520, 583737, 37),
 (7776, 26244, 38),
 (1024, 243, 12),
 (768, 7464, 40),
 (42461529762, 286699, 41),
 (768, 1, 3),
 (768, 4144, 43),
 (2080000, 1, 3),
 (24576, 519128, 45),
 (162, 32, 8),
 (1889568, 274624, 47),
 (1875, 572851622, 48),
 (512, 27141259, 49),
 (230496, 32, 8)

In [ ]:
#functions not used in the end
def standard_count(string, nums, preprocessing=True, midprocessing=False, killtest=False, print_info=False):#nums as list
    the_sum = 0
    #preprocessing: dots around max-sized #-sequences, extending #-sequences that are too small
    if preprocessing or midprocessing:
        absent = [n for n in range(1, max(nums)) if n not in nums]
    if preprocessing:
        while "?"+"#"*max(nums) in string or "#"*max(nums)+"?" in string:#the rest are tough to evaluate because of the many n in absent: or "?"+"#"*n+"." in string or "."+"#"*n+"?" in string:
            string = processround(string, nums, absent=absent)
      
    str_list = [string]
    if print_info:
        print("\r"+str(nums), end="                         ")
    while str_list != []: #list emptied when items have no "?" anymore
        string = str_list[-1]
        #remove old string
        str_list.remove(string)       
        if midprocessing:
            string = processround(string, nums, absent=absent)
        if killtest:#kill a string with an impossible number of # and ?
            if string.count("#") > sum(nums) or string.count("#")+string.count("?") < sum(nums):
                continue 

        test_list = [string.replace("?", ".", 1), string.replace("?", "#", 1)]
        for test_string in test_list:
            len_list = list(map(len,test_string[:test_string.find("?")].replace("."," ").split()))
            if len_list == nums[:len(len_list)] or (test_string[:test_string.find("?")][-1]=="#" and len_list[:-1] == nums[:len(len_list)][:-1] and len_list[-1] <= nums[len(len_list)-1]):
                if "?" in test_string:
                    str_list.append(test_string)
                elif list(map(len,test_string.replace("."," ").split())) == nums:  
                    the_sum = the_sum + 1
        if print_info:
            print(str_list)
    return the_sum

def stride_count(string, nums, stride=2, preprocessing=True, midprocessing=False, killtest=False, print_info=False):#nums as list
    the_sum = 0
    #preprocessing: dots around max-sized #-sequences, extending #-sequences that are too small
    if preprocessing or midprocessing:
        absent = [n for n in range(1, max(nums)) if n not in nums]
    if preprocessing:
        while "?"+"#"*max(nums) in string or "#"*max(nums)+"?" in string:#the rest are tough to evaluate because of the many n in absent: or "?"+"#"*n+"." in string or "."+"#"*n+"?" in string:
            string = processround(string, nums, absent=absent)
      
    str_list = [string]
    if print_info:
        print(str(nums), end="                         ")
    while str_list != []: #list emptied when items have no "?" anymore
        string = str_list[-1]
        #remove old string
        str_list.remove(string)       
        if midprocessing:
            string = processround(string, nums, absent=absent)
        if killtest:#kill a string with an impossible number of # and ?
            if string.count("#") > sum(nums) or string.count("#")+string.count("?") < sum(nums):
                continue 
        if stride==3 and string.count("?")>2:
            test_list = [string.replace("?", ".", 3), string.replace("?", ".", 2).replace("?", "#", 1), string.replace("?", ".", 1).replace("?", "#", 2), string.replace("?", ".", 1).replace("?", "#", 1).replace("?", ".", 1),
                          string.replace("?", "#", 1).replace("?", ".", 2), string.replace("?", "#", 2).replace("?", ".", 1), string.replace("?", "#", 3), string.replace("?", "#", 1).replace("?", ".", 1).replace("?", "#", 1)]
        if stride==2 and string.count("?")>1:
            test_list = [string.replace("?", ".", 2), string.replace("?", ".", 1).replace("?", "#", 1), string.replace("?", "#", 1).replace("?", ".", 1), string.replace("?", "#", 2)]
        else: #any other stride value goes here haha
            test_list = [string.replace("?", ".", 1), string.replace("?", "#", 1)]
        
        for test_string in test_list:
            len_list = list(map(len,test_string[:test_string.find("?")].replace("."," ").split()))
            if len_list == nums[:len(len_list)] or (test_string[:test_string.find("?")][-1]=="#" and len_list[:-1] == nums[:len(len_list)][:-1] and len_list[-1] <= nums[len(len_list)-1]):
                if "?" in test_string:
                    str_list.append(test_string)
                elif list(map(len,test_string.replace("."," ").split())) == nums:  
                    the_sum = the_sum + 1
        if print_info:
            print("\r"+string, end="                    ")
    return the_sum

def count_dotless(string, nums):#inefficient
    string = "." + string + "." #so that all interstitials are min 1
    total_dots = len(string)-sum(nums)
    max_possible_dots = len(string) - sum(nums) - len(nums) #max for each position, because of min=1 for each interstitium

    the_sum = 0
    interstitials = [[]] #list of interstitial values, added little by little

    while interstitials!=[]:
        previous = interstitials[-1].copy()
        #try all possible values for next interstitial
        for n_dots in range(1,min(max_possible_dots,total_dots-sum(previous))+1):
            attempted = previous + [n_dots]
            #check if valid
            for i in range(len(attempted)+1): #testing also using that the next has at least one dot
                st,end = sum(attempted[:i])+sum(nums[:i]) , sum((attempted+[1])[:i+1])+sum(nums[:i])
                if "#" in set(string[st:end]):
                    break
            else:#if passed all tests
                if len(attempted)==len(nums)+1 and sum(attempted)==total_dots:
                    the_sum = the_sum + 1 #either add to sum
                else: #or add to list to be further checked
                    interstitials.append(attempted)
        interstitials.remove(previous)
    return the_sum


In [ ]:
#in principle good, but slow
with open("exampleinput12.dat", 'r') as f:
    lines = f.readlines()

sum = 0
for line in lines:
    content = line.removesuffix('\n').split()
    string, nums = ((content[0]+'?')*5)[:-1], [int(_) for _ in content[1].split(sep=',')]*5
    #preprocessing: dots around max-sized #-sequences, extending #-sequences that are too small
    absent = [n for n in range(1, max(nums)) if n not in nums]
    while "?"+"#"*max(nums) in string or "#"*max(nums)+"?" in string or "?"+"#"*n+"." in string or "."+"#"*n+"?" in string:
        string = string.replace("?"+"#"*max(nums),"."+"#"*max(nums))
        string = string.replace("#"*max(nums)+"?","#"*max(nums)+".")
        for n in absent:
            string = string.replace("?"+"#"*n+".","#"*(n+1)+".")
            string = string.replace("."+"#"*n+"?","."+"#"*(n+1))     
    #creating list of strings with all possible subs, testing early        
    str_list = [string]
    while str_list != []: #list emptied when items have no "?" anymore
        string = str_list[-1]
        #remove old string
        str_list.remove(string)       
        #replace first ? by . and try it
        subbed_dot = string.replace("?", ".", 1)
        len_list = [len(_) for _ in subbed_dot[:subbed_dot.find("?")].split(sep=".") if _ != ""]
        if len_list == nums[:len(len_list)] or (len_list[:-1] == nums[:len(len_list)][:-1] and len_list[-1] <= nums[len(len_list)-1] and subbed_dot[:subbed_dot.find("?")][-1]=="#"):
            if "?" in subbed_dot:
                str_list.append(subbed_dot)
            elif [len(_) for _ in subbed_dot.split(sep=".") if _ != ""] == nums:
                sum = sum + 1
        #replace first ? by # and try it
        subbed_ttt = string.replace("?", "#", 1)
        len_list = [len(_) for _ in subbed_ttt[:subbed_ttt.find("?")].split(sep=".") if _ != ""]
        if len_list == nums[:len(len_list)] or (len_list[:-1] == nums[:len(len_list)][:-1] and len_list[-1] <= nums[len(len_list)-1] and subbed_ttt[:subbed_ttt.find("?")][-1]=="#"):
            if "?" in subbed_ttt:
                str_list.append(subbed_ttt)
            elif [len(_) for _ in subbed_ttt.split(sep=".") if _ != ""] == nums:
                sum = sum + 1
        # print(str_list)
sum

525152

.

.

.

.

.

.

.

.

.

In [ ]:
#in principle good, but takes too long
sum = 0
with open("exampleinput12.dat", 'r') as f:
    #reading file line
    f.readline();f.readline();f.readline()
    content = f.readline().removesuffix('\n').split()
    if True:#while content != []:
        # string, nums = ((content[0])), [int(_) for _ in content[1].split(sep=',')]
        string, nums = ((content[0]+'?')*5)[:-1], [int(_) for _ in content[1].split(sep=',')]*5
                
        #creating list of strings with all possible subs, testing early        
        str_list = [string]


        while "?" in str_list[0]: #assumes 0th element has the most "?"
            old_string = str_list[0]
            
            #replace first ? by . and try it
            subbed_dot = old_string.replace("?", ".", 1)
            len_list = [len(_) for _ in subbed_dot[:subbed_dot.find("?")].split(sep=".") if _ != ""]
            if len_list[:-1] == nums[:len(len_list)][:-1]:
                if len_list==[] or (len_list[-1] == nums[len(len_list)-1]) or (len_list[-1] <= nums[len(len_list)-1] and subbed_dot[:subbed_dot.find("?")][-1]=="#"):
                    str_list.append(subbed_dot)
            #replace first ? by # and try it
            subbed_ttt = old_string.replace("?", "#", 1)
            len_list = [len(_) for _ in subbed_ttt[:subbed_ttt.find("?")].split(sep=".") if _ != ""]
            if len_list[:-1] == nums[:len(len_list)][:-1]:
                if len_list==[] or (len_list[-1] == nums[len(len_list)-1]) or (len_list[-1] <= nums[len(len_list)-1] and subbed_ttt[:subbed_ttt.find("?")][-1]=="#"):
                # if len_list==[] or len_list[-1] <= nums[len(len_list)-1]:
                    str_list.append(subbed_ttt)
            #remove element 0
            str_list = str_list[1:]
            print(str_list)

        #testing how many possibilities match the numbers
        sum = sum + [[len(_) for _ in string.split(sep=".") if _ != ""] for string in str_list].count(nums)
        #reading file line in loop    
        content = f.readline().removesuffix('\n').split()
sum

['.???.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...', '#???.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...']
['#???.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...', '..??.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...', '.#??.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...']
['..??.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...', '.#??.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...', '##??.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...']
['.#??.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...', '##??.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...', '...?.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...', '..#?.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...']
['##??.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...', '...?.#...#...?????.#...#...?????.#...#...????

16